# Redes Neuronales

In [1]:
%matplotlib inline
import matplotlib.pyplot as plt
import numpy as np
import random
import statsmodels.api as sm
import statsmodels.stats.diagnostic as smd
import pandas as pd
from sklearn import datasets
from sklearn.model_selection import train_test_split
import seaborn as sns
import statsmodels.api as sm
import scipy.stats as stats
import statsmodels.stats.diagnostic as diag
from sklearn.preprocessing import StandardScaler
from sklearn.neural_network import MLPClassifier

from sklearn.compose import make_column_selector as selector #Para seleccionar de forma automática las variables numéricas y categóricas
from sklearn.preprocessing import OneHotEncoder #Para codificar las variables categóricas usando dummies
from sklearn.preprocessing import StandardScaler #Para normalizar las variables numéricas
from sklearn.compose import ColumnTransformer #Modifica las columnas usando los preprocesadores
from sklearn.pipeline import make_pipeline #Planifica una secuencia de procesos
from sklearn import set_config #Para mostrar graficamente el pipeline
from sklearn.model_selection import GridSearchCV

set_config(display='diagram')
#Metrics
from sklearn.metrics import make_scorer, accuracy_score,precision_score
from sklearn.metrics import classification_report
from sklearn.metrics import confusion_matrix
from sklearn.metrics import accuracy_score ,precision_score,recall_score,f1_score

In [2]:
# Abrir CSV
df2 = pd.read_csv('/dataset/cleaned_orders.csv')
df2['delivery_delayed'] =  df2['delivery_delayed'].astype(int)
df2['different_state'] = df2['different_state'].astype(int)

# Eliminar algunas variables respuesta de otros modelos
df2.drop(columns=['has_comment', 'has_title'], inplace=True)
df2 = df2.dropna()

In [3]:
print(df2.shape)
print(df2.columns)

(94528, 71)
Index(['estimated_delivery_days', 'delivery_days', 'delivery_delta',
       'delivery_delayed', 'avg_product_price', 'total_order_price',
       'item_count', 'order_freight_value', 'avg_product_weight',
       'total_order_weight_g', 'avg_product_volume_cm3',
       'total_order_volume_cm3', 'freight_to_price', 'shipping_distance_km',
       'different_state', 'payment_installments', 'split_payments',
       'review_score', 'seller_state_AM', 'seller_state_BA', 'seller_state_CE',
       'seller_state_DF', 'seller_state_ES', 'seller_state_GO',
       'seller_state_MA', 'seller_state_MG', 'seller_state_MS',
       'seller_state_MT', 'seller_state_PA', 'seller_state_PB',
       'seller_state_PE', 'seller_state_PI', 'seller_state_PR',
       'seller_state_RJ', 'seller_state_RN', 'seller_state_RO',
       'seller_state_RS', 'seller_state_SC', 'seller_state_SE',
       'seller_state_SP', 'customer_state_AC', 'customer_state_AL',
       'customer_state_AM', 'customer_state_AP', '

In [4]:
#la variable respuesta
target = df2.pop('review_score')

In [5]:
#Creamos los selectores
selector_numericas = selector(dtype_exclude=object) #Seleccionamos todas las variables que no son Object
selector_categoricas = selector(dtype_include=object) #Seleccionamos todas las varibles de tipo object
#Seleccionamos las columnas que cumplen con los selectores pasándole los datos
numericas = selector_numericas(df2)
categoricas = selector_categoricas(df2)
#veamos que tan bien separó las variables
print("Numéricas: ",numericas)
print("Categóricas: ", categoricas)

Numéricas:  ['estimated_delivery_days', 'delivery_days', 'delivery_delta', 'delivery_delayed', 'avg_product_price', 'total_order_price', 'item_count', 'order_freight_value', 'avg_product_weight', 'total_order_weight_g', 'avg_product_volume_cm3', 'total_order_volume_cm3', 'freight_to_price', 'shipping_distance_km', 'different_state', 'payment_installments', 'split_payments', 'seller_state_AM', 'seller_state_BA', 'seller_state_CE', 'seller_state_DF', 'seller_state_ES', 'seller_state_GO', 'seller_state_MA', 'seller_state_MG', 'seller_state_MS', 'seller_state_MT', 'seller_state_PA', 'seller_state_PB', 'seller_state_PE', 'seller_state_PI', 'seller_state_PR', 'seller_state_RJ', 'seller_state_RN', 'seller_state_RO', 'seller_state_RS', 'seller_state_SC', 'seller_state_SE', 'seller_state_SP', 'customer_state_AC', 'customer_state_AL', 'customer_state_AM', 'customer_state_AP', 'customer_state_BA', 'customer_state_CE', 'customer_state_DF', 'customer_state_ES', 'customer_state_GO', 'customer_state_

In [6]:
preprocesador_numerico = StandardScaler()

In [7]:
preprocesador = ColumnTransformer([
    ('standard_scaler',preprocesador_numerico, numericas)
])

In [8]:
modelo = make_pipeline(preprocesador,MLPClassifier(activation="tanh",verbose=True,hidden_layer_sizes=(5,3)))
modelo

Pipeline(steps=[('columntransformer',
                 ColumnTransformer(transformers=[('standard_scaler',
                                                  StandardScaler(),
                                                  ['estimated_delivery_days',
                                                   'delivery_days',
                                                   'delivery_delta',
                                                   'delivery_delayed',
                                                   'avg_product_price',
                                                   'total_order_price',
                                                   'item_count',
                                                   'order_freight_value',
                                                   'avg_product_weight',
                                                   'total_order_weight_g',
                                                   'avg_product_volume_cm3',
                                                   'total_order_volume_cm...
                                                   'split_payments',
                                                   'seller_state_AM',
                                                   'seller_state_BA',
                                                   'seller_state_CE',
                                                   'seller_state_DF',
                                                   'seller_state_ES',
                                                   'seller_state_GO',
                                                   'seller_state_MA',
                                                   'seller_state_MG',
                                                   'seller_state_MS',
                                                   'seller_state_MT',
                                                   'seller_state_PA',
                                                   'seller_state_PB',
                                                   'seller_state_PE', ...])])),
                ('mlpclassifier',
                 MLPClassifier(activation='tanh', hidden_layer_sizes=(5, 3),
                               verbose=True))])

In [9]:
data_train, data_test, target_train, target_test = train_test_split(
    df2, target, train_size=0.7,test_size=0.3)
print("Shape train",data_train.shape)
print("Shape test",data_test.shape)
df2.isnull().sum()

Shape train (66169, 70)
Shape test (28359, 70)


,0
estimated_delivery_days,0
delivery_days,0
delivery_delta,0
delivery_delayed,0
avg_product_price,0
...,...
customer_state_TO,0
payment_type_boleto,0
payment_type_credit_card,0
payment_type_debit_card,0


In [ ]:
modelo.fit(data_train,target_train)

Iteration 1, loss = 1.24629281
Iteration 2, loss = 1.12370121
Iteration 3, loss = 1.10841385
Iteration 4, loss = 1.10151907
Iteration 5, loss = 1.09662183
Iteration 6, loss = 1.09365907
Iteration 7, loss = 1.09225288
Iteration 8, loss = 1.09131412
Iteration 9, loss = 1.09059774
Iteration 10, loss = 1.09011563
Iteration 11, loss = 1.08944890
Iteration 12, loss = 1.08889130
Iteration 13, loss = 1.08839922
Iteration 14, loss = 1.08780547
Iteration 15, loss = 1.08738890
Iteration 16, loss = 1.08693496
Iteration 17, loss = 1.08646877
Iteration 18, loss = 1.08594029
Iteration 19, loss = 1.08541271
Iteration 20, loss = 1.08511844
Iteration 21, loss = 1.08456588
Iteration 22, loss = 1.08424568
Iteration 23, loss = 1.08397160
Iteration 24, loss = 1.08365843
Iteration 25, loss = 1.08321529
Iteration 26, loss = 1.08308884
Iteration 27, loss = 1.08293917
Iteration 28, loss = 1.08266435
Iteration 29, loss = 1.08243795
Iteration 30, loss = 1.08227581
Iteration 31, loss = 1.08209433
Iteration 32, los

Pipeline(steps=[('columntransformer',
                 ColumnTransformer(transformers=[('standard_scaler',
                                                  StandardScaler(),
                                                  ['estimated_delivery_days',
                                                   'delivery_days',
                                                   'delivery_delta',
                                                   'delivery_delayed',
                                                   'avg_product_price',
                                                   'total_order_price',
                                                   'item_count',
                                                   'order_freight_value',
                                                   'avg_product_weight',
                                                   'total_order_weight_g',
                                                   'avg_product_volume_cm3',
                                                   'total_order_volume_cm...
                                                   'split_payments',
                                                   'seller_state_AM',
                                                   'seller_state_BA',
                                                   'seller_state_CE',
                                                   'seller_state_DF',
                                                   'seller_state_ES',
                                                   'seller_state_GO',
                                                   'seller_state_MA',
                                                   'seller_state_MG',
                                                   'seller_state_MS',
                                                   'seller_state_MT',
                                                   'seller_state_PA',
                                                   'seller_state_PB',
                                                   'seller_state_PE', ...])])),
                ('mlpclassifier',
                 MLPClassifier(activation='tanh', hidden_layer_sizes=(5, 3),
                               verbose=True))])

In [ ]:
pred = modelo.predict(data_test)
pred[:5]

array([5., 5., 5., 5., 5.])

In [ ]:
cm = confusion_matrix(target_test,pred)
accuracy=accuracy_score(target_test,pred)
precision =precision_score(target_test,pred,average='micro')
recall =  recall_score(target_test,pred,average='micro')
f1 = f1_score(target_test,pred,average='micro')
print('Matriz de confusión\n',cm)
print('Accuracy: ',accuracy)
print('Precision: ', precision)
print('recall: ',recall)

Matriz de confusión
 [[  965     0     0     0  1831]
 [  143     0     0     0   740]
 [  158     0     0     0  2173]
 [  116     0     0     0  5462]
 [  172     0     0     0 16599]]
Accuracy:  0.6193448288021439
Precision:  0.6193448288021439
recall:  0.6193448288021439



#### Tuneo de parámetros usando grid search cross validation

Con la función get_params del clasificador podemos saber cuales fueron los nombres y valores de los parámetros del modelo

In [ ]:
modelo.get_params().keys()

dict_keys(['memory', 'steps', 'transform_input', 'verbose', 'columntransformer', 'mlpclassifier', 'columntransformer__force_int_remainder_cols', 'columntransformer__n_jobs', 'columntransformer__remainder', 'columntransformer__sparse_threshold', 'columntransformer__transformer_weights', 'columntransformer__transformers', 'columntransformer__verbose', 'columntransformer__verbose_feature_names_out', 'columntransformer__standard_scaler', 'columntransformer__standard_scaler__copy', 'columntransformer__standard_scaler__with_mean', 'columntransformer__standard_scaler__with_std', 'mlpclassifier__activation', 'mlpclassifier__alpha', 'mlpclassifier__batch_size', 'mlpclassifier__beta_1', 'mlpclassifier__beta_2', 'mlpclassifier__early_stopping', 'mlpclassifier__epsilon', 'mlpclassifier__hidden_layer_sizes', 'mlpclassifier__learning_rate', 'mlpclassifier__learning_rate_init', 'mlpclassifier__max_fun', 'mlpclassifier__max_iter', 'mlpclassifier__momentum', 'mlpclassifier__n_iter_no_change', 'mlpclass

In [ ]:
parametros_tun  = param_grid = {
    'mlpclassifier__max_iter': (100,200,500,1000),
    'mlpclassifier__learning_rate_init': (0.01, 0.1, 1,10),
    'mlpclassifier__hidden_layer_sizes':((3,2),(5,3),(10,2))}
model_grid_search = GridSearchCV(modelo, param_grid=parametros_tun,
                                 n_jobs=2, cv=10) #Vamos a usar dos procesadores(n_jobs), y 10 k-folds
model_grid_search.fit(data_train, target_train)

Iteration 1, loss = 1.12957418
Iteration 2, loss = 1.09499761
Iteration 3, loss = 1.09352466
Iteration 4, loss = 1.09167359
Iteration 5, loss = 1.09135421
Iteration 6, loss = 1.09028010
Iteration 7, loss = 1.09003749
Iteration 8, loss = 1.09013326
Iteration 9, loss = 1.08970929
Iteration 10, loss = 1.08955815
Iteration 11, loss = 1.08902554
Iteration 12, loss = 1.08897353
Iteration 13, loss = 1.08912727
Iteration 14, loss = 1.08863666
Iteration 15, loss = 1.08858610
Iteration 16, loss = 1.08884826
Iteration 17, loss = 1.08810210
Iteration 18, loss = 1.08846702
Iteration 19, loss = 1.08815484
Iteration 20, loss = 1.08830856
Iteration 21, loss = 1.08766656
Iteration 22, loss = 1.08795475
Iteration 23, loss = 1.08792174
Iteration 24, loss = 1.08779311
Iteration 25, loss = 1.08839731
Iteration 26, loss = 1.08773235
Iteration 27, loss = 1.08777899
Iteration 28, loss = 1.08800372
Iteration 29, loss = 1.08798966
Iteration 30, loss = 1.08777069
Iteration 31, loss = 1.08765015
Iteration 32, los

GridSearchCV(cv=10,
             estimator=Pipeline(steps=[('columntransformer',
                                        ColumnTransformer(transformers=[('standard_scaler',
                                                                         StandardScaler(),
                                                                         ['estimated_delivery_days',
                                                                          'delivery_days',
                                                                          'delivery_delta',
                                                                          'delivery_delayed',
                                                                          'avg_product_price',
                                                                          'total_order_price',
                                                                          'item_count',
                                                                          'order_freight_value',
                                                                          'avg_product_weight',
                                                                          'total_order_weight_g',
                                                                          'avg_product_volum...
                                                                          'seller_state_MS',
                                                                          'seller_state_MT',
                                                                          'seller_state_PA',
                                                                          'seller_state_PB',
                                                                          'seller_state_PE', ...])])),
                                       ('mlpclassifier',
                                        MLPClassifier(activation='tanh',
                                                      hidden_layer_sizes=(5, 3),
                                                      verbose=True))]),
             n_jobs=2,
             param_grid={'mlpclassifier__hidden_layer_sizes': ((3, 2), (5, 3),
                                                               (10, 2)),
                         'mlpclassifier__learning_rate_init': (0.01, 0.1, 1,
                                                               10),
                         'mlpclassifier__max_iter': (100, 200, 500, 1000)})

¿Cuál será la mejor combinación de parámetros?


In [ ]:
model_grid_search.best_params_

{'mlpclassifier__hidden_layer_sizes': (3, 2),
 'mlpclassifier__learning_rate_init': 0.01,
 'mlpclassifier__max_iter': 200}

In [ ]:
parametros_tun  = param_grid = {
    'mlpclassifier__max_iter': ([200]),
    'mlpclassifier__learning_rate_init': ([0.01]),
    'mlpclassifier__hidden_layer_sizes':([(3,2)])}
model_grid_search = GridSearchCV(modelo, param_grid=parametros_tun,
                                 n_jobs=2, cv=10) #Vamos a usar dos procesadores(n_jobs), y 10 k-folds
model_grid_search.fit(data_train, target_train)

Iteration 1, loss = 1.16796088
Iteration 2, loss = 1.09914517
Iteration 3, loss = 1.09577918
Iteration 4, loss = 1.09351610
Iteration 5, loss = 1.09209949
Iteration 6, loss = 1.09120735
Iteration 7, loss = 1.09077835
Iteration 8, loss = 1.09000440
Iteration 9, loss = 1.09009234
Iteration 10, loss = 1.09008732
Iteration 11, loss = 1.08979246
Iteration 12, loss = 1.08968316
Iteration 13, loss = 1.08913036
Iteration 14, loss = 1.08968056
Iteration 15, loss = 1.08950610
Iteration 16, loss = 1.08899198
Iteration 17, loss = 1.08888880
Iteration 18, loss = 1.08891505
Iteration 19, loss = 1.08962051
Iteration 20, loss = 1.08914072
Iteration 21, loss = 1.08932710
Iteration 22, loss = 1.08878418
Iteration 23, loss = 1.08925082
Iteration 24, loss = 1.08899754
Iteration 25, loss = 1.08892128
Iteration 26, loss = 1.08893537
Iteration 27, loss = 1.08896084
Iteration 28, loss = 1.08889462
Iteration 29, loss = 1.08894999
Iteration 30, loss = 1.08937615
Iteration 31, loss = 1.08922276
Iteration 32, los

GridSearchCV(cv=10,
             estimator=Pipeline(steps=[('columntransformer',
                                        ColumnTransformer(transformers=[('standard_scaler',
                                                                         StandardScaler(),
                                                                         ['estimated_delivery_days',
                                                                          'delivery_days',
                                                                          'delivery_delta',
                                                                          'delivery_delayed',
                                                                          'avg_product_price',
                                                                          'total_order_price',
                                                                          'item_count',
                                                                          'order_freight_value',
                                                                          'avg_product_weight',
                                                                          'total_order_weight_g',
                                                                          'avg_product_volum...
                                                                          'seller_state_MA',
                                                                          'seller_state_MG',
                                                                          'seller_state_MS',
                                                                          'seller_state_MT',
                                                                          'seller_state_PA',
                                                                          'seller_state_PB',
                                                                          'seller_state_PE', ...])])),
                                       ('mlpclassifier',
                                        MLPClassifier(activation='tanh',
                                                      hidden_layer_sizes=(5, 3),
                                                      verbose=True))]),
             n_jobs=2,
             param_grid={'mlpclassifier__hidden_layer_sizes': [(3, 2)],
                         'mlpclassifier__learning_rate_init': [0.01],
                         'mlpclassifier__max_iter': [200]})

Vamos a predecir con el mejor modelo

In [ ]:
pred = model_grid_search.predict(data_test)

In [ ]:
cm = confusion_matrix(target_test,pred)
accuracy=accuracy_score(target_test,pred)
precision =precision_score(target_test,pred,average='micro')
recall =  recall_score(target_test,pred,average='micro')
f1 = f1_score(target_test,pred,average='micro')
print('Matriz de confusión\n',cm)
print('Accuracy: ',accuracy)
print('Precision: ', precision)
print('recall: ',recall)

Matriz de confusión
 [[  967     0     0    12  1817]
 [  145     0     0     2   736]
 [  162     0     0     1  2168]
 [  117     0     0     6  5455]
 [  175     0     0    14 16582]]
Accuracy:  0.619027469233753
Precision:  0.619027469233753
recall:  0.619027469233753
